In [ ]:
import numpy as np
import scipy.sparse as scs
from scipy.stats import multinomial
import pathlib as pl
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import mmread

import umap as um


from os import listdir
from os.path import isfile, join
import os

import random

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import seaborn as sns

from sklearn.preprocessing import StandardScaler

import random
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, average_precision_score


import tensorflow as tf

import os
import pickle

import seaborn as sns

from sklearn.model_selection import train_test_split

from kneed import KneeLocator

from numpy.random import seed

import time


In [ ]:
from plot_results import get_dbl_metrics
from vae import define_clust_vae
from PU import PU, epoch_PU, epoch_PU2
from classifier import define_classifier
from mk_doublets import sim_inflate, sim_avg, sim_sum
from cluster import cluster, fast_cluster
from vaeda import vaeda

In [ ]:
import scanpy as sc
import anndata

In [ ]:
import numpy as np
import scipy.sparse as scs
from scipy.stats import multinomial
import pathlib as pl
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import mmread

import umap as um


from os import listdir
from os.path import isfile, join
import os

import random

from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import seaborn as sns

from sklearn.preprocessing import StandardScaler

import random
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, average_precision_score


import tensorflow as tf

import os
import pickle

import seaborn as sns

from sklearn.model_selection import train_test_split

from kneed import KneeLocator

from numpy.random import seed

import time
import scanpy as sc
import anndata




from plot_results import get_dbl_metrics
from vae import define_clust_vae
from PU import PU, epoch_PU, epoch_PU2
from classifier import define_classifier
from mk_doublets import sim_inflate, sim_avg, sim_sum
from cluster import cluster, fast_cluster






In [ ]:
def sigmoid(x):
    sig = 1 / (1 + np.exp(-(12*x)+6))
    return sig

In [ ]:
data_dir = 'sce_normalized_data_inflate'
save = True
use_old_doubs=False
remove_homos = True
projections = False

#VAE hyperparams
eps  = 1000
enc_sze = 5
use_old=False
pat = 20
LR=1e-3
clust_weight = 10000

#PU hyperparameters
cls_eps = 250 
stop_metric = 'ValAUC'
puPat = 5
puLR =1e-3
pu_num_layers = 1
k_mult = 2
N = 10

In [ ]:
seeds = [42, 29503, 432809, 42, 132975, 9231996, 12883823, 9231996, 1234, 62938, 57203 ,109573, 23]


In [ ]:
data_dir = 'sce_normalized_data_inflate'
path = '../data/' + data_dir + '/'
files = [f for f in listdir(path) if (isfile(join(path, f)) & (f[-18:-4] == 'real_logcounts'))]
files = np.sort(files)
files = files#[:2]
#files = ['nuc-MULTI_real_logcounts.mtx', 'pbmc-ch_real_logcounts.mtx', 'hm-12k_real_logcounts.mtx']
files

In [ ]:
#files = files[9:]
#files

#nohup jupyter nbconvert --to notebook --execute VAEDA.ipynb

In [ ]:
time_names = ['total', 'simulation', 'HVGs', 'scaling1', 'knn', 'downsample', 'scaling2', 'cluster', 'vae', 'epoch_selection', 'PU_loop']

In [ ]:
N=1
use_old=False
remove_homos = True

k_mult = 2

In [ ]:
#from vaeda import vaeda

In [ ]:
#vadea = all default but PCA is 30
#vadea2 = all default but PCA is 30, and ClustWeight is 20000
#vadea3 = all default but PCA is 40, and ClustWeight is 20000
#vadea4 = all default but PCA is 30, and ClustWeight is 30000 
#vadea5 = all default but PCA is 30, and ClustWeight is 20000, enc_sze=3 
#vadea6 = all default but PCA is 30, and ClustWeight is 20000, numhvg=1000 *

In [ ]:
'''- clust weight = 30000
- enc_sze=3
- numhvg=1000'''

In [ ]:
knees = []
for file in files:
    
    data_name = file[:-19]
    print(data_name) 
    
    save_path = '../results_PU/final_vaeda_result/VAEDA_new_calls'
    #save_path = '../results_PU/doublet_calls/vaeda'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path = save_path + '/' + data_name + '_'
    print(save_path)

    real_path = '../data/mtx_files/' + data_name + '.mtx'
    ano_path  = '../data/mtx_files/' + data_name + '_anno.csv'
    
    print('loading in real mtx')
    dat_real = mmread(real_path)
    Xr = scs.csr_matrix(dat_real).toarray().T
    
    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp

    
    preds, preds_on_P, calls, encoding, knn_features = vaeda(Xr, save_dir=save_path, use_old=use_old, pca_comp=30, clust_weight=20000)
    
    #X, save_dir='',
    #gene_thresh=.01, num_hvgs=2000,
    #pca_comp=30, quant=0.25,
    #enc_sze=5, max_eps_vae=1000, pat_vae=20, LR_vae=1e-3, clust_weight=20000, rate=-0.75,
    #N=1, k_mult=2, max_eps_PU=250, LR_PU=1e-3,
    #remove_homos=True, use_old=True,
    #seeds=[42, 29503, 432809, 42, 132975, 9231996, 12883823, 9231996, 1234, 62938, 57203 ,109573, 23])
    
    true = np.concatenate([true, np.full(len(preds_on_P),2)])
    labels = np.concatenate([labels, np.full(len(preds_on_P),'simulated')])
    
    #RESULTS
    preds_sing = preds[true[true<2]==0]
    preds_doub_test = preds[true[true<2]==1]
    preds_doub_train = preds_on_P
    labs = ['singlet', 'actual doublet', 'simulated doublet']
    cols = np.concatenate([preds, preds_on_P])

    #SAVE SCORES
    tmp1 = np.zeros((len(preds), 3))
    df = pd.DataFrame(tmp1, columns=['ano', 'doublet_scores', 'predicted_doublets'])
    df.ano = true[true<2]
    df.doublet_scores = preds
    df.predicted_doublets = calls
    if(save):
        df.to_csv(save_path + 'vaeda3_scores_1.csv') 

    #PR and ROC curves
    plt.figure(4)
    res = get_dbl_metrics(true[true<2], preds)
    plt.show()
    plt.close()

    #save AUCs
    hm_pr = pd.DataFrame(np.array(res).T, index=['AUROC', 'AUPRC', 'AP']).T
    if(save):
        hm_pr.to_csv(save_path + 'scores_ROC_PR_area_ALL.csv') 
        
        
    if(projections):
        proj_enc = um.UMAP(n_neighbors=7).fit_transform(encoding)

        fig, ax = plt.subplots()
        cols = ['r', 'b', 'g']
        size= (1/np.sqrt(encoding.shape[0]))*30
        for i in [0,2,1]:
            scatter = ax.scatter(proj_enc[true==i,0],proj_enc[true==i,1], s=size, alpha=0.5, label=str(i), cmap=plt.cm.Spectral)
        #scatter = ax.scatter(proj_enc[:,0],proj_enc[:,1], s=size, alpha=1, c=true, cmap='twilight')
        #plt.legend()

        plt.title(data_name + ' encoding UMAP')
        plt.legend()
        #make square
        x0,x1 = ax.get_xlim()
        y0,y1 = ax.get_ylim()
        ax.set_aspect(abs(x1-x0)/abs(y1-y0))
        #remove ticks
        plt.tick_params(axis='both', 
                        which='both', 
                        bottom=False, 
                        top=False, 
                        labelbottom=False, 
                        right=False, 
                        left=False, 
                        labelleft=False)
        #if save:
        #    plt.savefig(fig_path + 'UMAP.png', dpi=100)
        plt.show()
        plt.close(fig)
        
        
        fig, ax = plt.subplots()
        cols = ['r', 'b', 'g']
        size= (1/np.sqrt(encoding.shape[0]))*30

        p = proj_enc[true<2,:]
        p = p[np.argsort(preds)[::-1],:]
        scatter = ax.scatter(p[:,0],p[:,1], s=size, alpha=1, c=preds[np.argsort(preds)[::-1]], cmap='viridis')
        fig.colorbar(mappable=scatter)

        plt.title(data_name + ' encoding UMAP')
        plt.legend()
        #make square
        x0,x1 = ax.get_xlim()
        y0,y1 = ax.get_ylim()
        ax.set_aspect(abs(x1-x0)/abs(y1-y0))
        #remove ticks
        plt.tick_params(axis='both', 
                        which='both', 
                        bottom=False, 
                        top=False, 
                        labelbottom=False, 
                        right=False, 
                        left=False, 
                        labelleft=False)
        #if save:
        #    plt.savefig(fig_path + 'UMAP.png', dpi=100)
        plt.show()
        plt.close(fig)

    
        proj_enc = um.UMAP(n_neighbors=7).fit_transform(encoding[true<2,:])

        fig, ax = plt.subplots()
        cols = ['r', 'b', 'g']
        size= (1/np.sqrt(encoding.shape[0]))*30
        for i in ['singlet', 'doublet']:
            scatter = ax.scatter(proj_enc[calls==i,0],proj_enc[calls==i,1], s=size, alpha=0.5, label=str(i), cmap=plt.cm.Spectral)
        #scatter = ax.scatter(proj_enc[:,0],proj_enc[:,1], s=size, alpha=1, c=true, cmap='twilight')
        #plt.legend()

        plt.title(data_name + ' encoding UMAP Doublet Calls')
        plt.legend()
        #make square
        x0,x1 = ax.get_xlim()
        y0,y1 = ax.get_ylim()
        ax.set_aspect(abs(x1-x0)/abs(y1-y0))
        #remove ticks
        plt.tick_params(axis='both', 
                        which='both', 
                        bottom=False, 
                        top=False, 
                        labelbottom=False, 
                        right=False, 
                        left=False, 
                        labelleft=False)
        #if save:
        #    plt.savefig(fig_path + 'UMAP.png', dpi=100)
        plt.show()
        plt.close(fig)
        

In [ ]:
clusters_real = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_clusters_real.npy')
clusters_sim = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_clusters_sim.npy')
emb_real = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_embedding_real.npy')
emb_sim = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_embedding_sim.npy')
df_scores = pd.read_csv('../results_PU/final_vaeda_result/VAEDA/hm-12k_scores_ROC_PR_area_ALL.csv')
knn_feat_real = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_knn_feature_real.npy')
knn_feat_sim = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_knn_feature_sim.npy')
sim_scores = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_scores_on_sim.npy')
scores = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_scores.npy')
sim_ind = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_sim_ind.npy')
time = pd.read_csv('../results_PU/final_vaeda_result/VAEDA/hm-12k_time.csv')
dat_sim = scs.load_npz('../results_PU/final_vaeda_result/VAEDA/hm-12k_sim_doubs.npz')
sim_which = np.load('../results_PU/final_vaeda_result/VAEDA/hm-12k_which_sim_doubs.npy')

In [ ]:
clusters_real2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_clusters_real.npy')
clusters_sim2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_clusters_sim.npy')
emb_real2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_embedding_real.npy')
emb_sim2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_embedding_sim.npy')
df_scores2 = pd.read_csv('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_scores_ROC_PR_area_ALL.csv')
knn_feat_real2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_knn_feature_real.npy')
knn_feat_sim2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_knn_feature_sim.npy')
sim_scores2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_scores_on_sim.npy')
scores2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_scores.npy')
sim_ind2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_sim_ind.npy')
time2 = pd.read_csv('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_time.csv')
dat_sim2 = scs.load_npz('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_sim_doubs.npz')
sim_which2 = np.load('../results_PU/final_vaeda_result/hyper_PcaComp30_ClustWeight20000/hm-12k_which_sim_doubs.npy')

In [ ]:
m = np.all(dat_sim.toarray()==dat_sim2.toarray())

In [ ]:
m

In [ ]:
np.all(sim_which==sim_which2)

In [ ]:
np.all(sim_ind==sim_ind2)

In [ ]:
np.all(clusters_real==clusters_real2)

In [ ]:
np.all(knn_feat_real2==knn_feat_real)

In [ ]:
knn_feat_real2

In [ ]:
knn_feat_real

In [ ]:
clusters_real = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_clusters_real.npy')
clusters_sim = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_clusters_sim.npy')
emb_real = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_embedding_real.npy')
emb_sim = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_embedding_sim.npy')
df_scores = pd.read_csv('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_scores_ROC_PR_area_ALL.csv')
knn_feat_real = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_knn_feature_real.npy')
knn_feat_sim = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_knn_feature_sim.npy')
sim_scores = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_scores_on_sim.npy')
scores = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_scores.npy')
sim_ind = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_sim_ind.npy')
time = pd.read_csv('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_time.csv')
dat_sim = scs.load_npz('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_sim_doubs.npz')
sim_which = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after2/cline-ch_which_sim_doubs.npy')

In [ ]:
clusters_real2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_clusters_real.npy')
clusters_sim2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_clusters_sim.npy')
emb_real2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_embedding_real.npy')
emb_sim2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_embedding_sim.npy')
df_scores2 = pd.read_csv('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_scores_ROC_PR_area_ALL.csv')
knn_feat_real2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_knn_feature_real.npy')
knn_feat_sim2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_knn_feature_sim.npy')
sim_scores2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_scores_on_sim.npy')
scores2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_scores.npy')
sim_ind2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_sim_ind.npy')
time2 = pd.read_csv('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_time.csv')
dat_sim2 = scs.load_npz('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_sim_doubs.npz')
sim_which2 = np.load('../results_PU/final_vaeda_result/fourtysix_filter_after/cline-ch_which_sim_doubs.npy')

In [ ]:
np.all(dat_sim.toarray()==dat_sim2.toarray())


In [ ]:
np.all(knn_feat_real==knn_feat_real2)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score




print(f1_score(labels[true<2], calls, average="macro"))
print(precision_score(labels[true<2], calls, average="macro"))
print(recall_score(labels[true<2], calls, average="macro"))
print(accuracy_score(labels[true<2], calls))
print()
print(f1_score(labels[true<2], calls, average="weighted"))
print(precision_score(labels[true<2], calls, average="weighted"))
print(recall_score(labels[true<2], calls, average="weighted"))
print(accuracy_score(labels[true<2], calls))
print()
print(f1_score(labels[true<2], calls, pos_label='doublet', average="binary"))
print(precision_score(labels[true<2], calls, pos_label='doublet', average="binary"))
print(recall_score(labels[true<2], calls,pos_label='doublet', average="binary"))
print(accuracy_score(labels[true<2], calls))
print()
print(f1_score(labels[true<2], calls, average="micro"))
print(precision_score(labels[true<2], calls, average="micro"))
print(recall_score(labels[true<2], calls, average="micro"))
print(accuracy_score(labels[true<2], calls))

In [ ]:
clusters_real = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_clusters_real.npy')
clusters_sim = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_clusters_sim.npy')
emb_real = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_embedding_real.npy')
emb_sim = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_embedding_sim.npy')
df_scores = pd.read_csv('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_scores_ROC_PR_area_ALL.csv')
knn_feat_real = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_knn_feature_real.npy')
knn_feat_sim = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_knn_feature_sim.npy')
sim_scores = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_scores_on_sim.npy')
scores = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_scores.npy')
sim_ind = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_sim_ind.npy')
time = pd.read_csv('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_time.csv')
dat_sim = scs.load_npz('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_sim_doubs.npz')
sim_which = np.load('../results_PU/final_vaeda_result/fourtythree_same_as_42/J293t-dm_which_sim_doubs.npy')


In [ ]:
sim_which.shape

In [ ]:
clusters_real.shape

In [ ]:
clusters_sim.shape

In [ ]:
emb_real.shape

In [ ]:
emb_sim.shape

In [ ]:
df_scores

In [ ]:
knn_feat_real.shape

In [ ]:
knn_feat_sim.shape

In [ ]:
sim_scores.shape

In [ ]:
scores.shape

In [ ]:
sim_ind.shape

In [ ]:
time

In [ ]:
dat_sim.shape

In [ ]:
for file in files:

    data_name = file[:-19]
    print(data_name) 
    
    save_path = '../results_PU/final_vaeda_result/eighteen_fast_clust/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path = save_path + '/' 
    print(save_path)

    fast = pd.read_csv(save_path + data_name + '_time.csv', index_col=0)
    
    
    save_path = '../results_PU/final_vaeda_result/seventeen_new_doubs/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    save_path = save_path + '/' 
    print(save_path)

    slow = (pd.read_csv(save_path + data_name + '_time.csv', index_col=0))
    
    print(slow-fast)

    

to speed up computational time

1.  tf.layers.batch_normalization(
    input_layer, fused=True, data_format=‘NCHW’) <- fused batch normilization
2. tf.ConfigProto, is used to set these configurations by passing to in the config attribute of tf.Session. For both parallelism configurations, is initialised as zero, they will revert to the number of logical CPU cores.
3. As you mentionned batch_size is really important to tune, it can lead to impressive speedup but check that your perplexity keeps relevant.
4. Most importantly, using sampled softmax instead of regular softmax is way faster. This would require you to use a [config.vocab_size, config.hidden_size] weight matrix instead of you [config.hidden_size, config.vocab_size]
6. tf.compat.v1.disable_eager_execution()
7. tf.config.optimizer.set_jit(True)


TRY #6 for sure!

In [ ]:
print('done')